In [ ]:
import sys, os
from pathlib import Path
cwd = os.getcwd()
parent = str(Path(cwd).parent)
sys.path.append(parent)

import torch
import gpytorch
from gpytorch.kernels import RBFKernel
from kernels.experimental_uncertainty_kernel import ExperimentalUncertaintyKernel, fix_lengthscale
from copy import deepcopy

SRBFKernel = fix_lengthscale(RBFKernel)

base_kernel = SRBFKernel()

euk = ExperimentalUncertaintyKernel(base_kernel, 5, (1,2,3), outputscale_fn=True, lengthscale_fn=True)
euk.initialize(os_v = torch.Tensor([0.0, 0.1,0.2]))
euk.initialize(ls_v = torch.Tensor([0.0, 0.1,0.2]))

x1 = torch.randn(10,4)
c = euk(x1, x1)
print(c.to_dense())
